In [192]:
from collections import defaultdict, deque
import numpy as np
import pandas as pd
from math import factorial

nodes_list = np.array([[ 1,  2], [ 2,  3], [ 3,  4], [ 4,  5], [ 5,  1],
                       [ 6,  7], [ 7,  8], [ 8,  9], [ 9, 10], [10, 11],
                       [11, 12], [12, 13], [13, 14], [14, 15], [15,  6],
                       [16, 17], [17, 18], [18, 19], [19, 20], [20, 16],
                       [15, 16], [ 7, 17], [ 9, 18], [11, 19], [13, 20],
                       [ 1,  8], [ 2, 10], [ 3, 12], [ 4, 14], [ 5,  6]])
dir_path = r'C:\Users\RedmiBook\Documents\SCE\third_1\Ann\\'

In [6]:
def build_start_graph():    
    graph = defaultdict(set)
    for edge in nodes_list:
        a, b = edge[0], edge[1]
        graph[a].add(b)
        graph[b].add(a)
    return graph

In [7]:
def check_way(res_graph, a, b):
    que_for_check = deque(res_graph[a])
    res_set = {a}
    checked_node = {a}
    while que_for_check and b not in res_set:
        elem = que_for_check.popleft() 
        if elem not in checked_node: 
            res_set = res_set | res_graph[elem]
            que_for_check.extend(list(res_graph[elem]))
            checked_node = checked_node | {elem}
    return b in res_set

In [8]:
def calc_f(M, a, b, c):
    res_arr = np.zeros(30)
    for _ in range(M):
        res_s = np.random.permutation(nodes_list)
        st_g = build_start_graph()    
        for i_r, rib in enumerate(res_s):
            st_g[rib[0]].remove(rib[1])
            st_g[rib[1]].remove(rib[0])
            if not check_way(st_g, a, b) or not check_way(st_g, c, b):
                finish = i_r
                break
        res_arr[finish] += 1
    return res_arr / M

In [124]:
def calc_F(p, n=30):
    f_n = factorial(n)
    q = 1 - p
    F = []
    for k in range(1, n + 1):
        res_sum = 0
        for i in range(k, n + 1):
            res_sum += (f_n / (factorial(i) * factorial(n - i))) * ((q) ** i) * ((p) ** (n - i))
        F.append(res_sum)
    return F

In [13]:
def build_graph_monte(p=0.5):    
    edges = (nodes_list.T * (np.random.rand(30) < [p]).astype(int)).T
    graph = defaultdict(set)
    for edge in edges:
        a, b = edge[0], edge[1]
        graph[a].add(b)
        graph[b].add(a)
    return graph

In [14]:
def count_res_monte(a, b, c, p=0.3, N=1000):
    positive = 0
    for _ in range(N):
        res_graph = build_graph_monte(p)
        if  check_way(res_graph, a, b) and check_way(res_graph, a, c):
            positive += 1
    return positive / N

In [193]:
N = 30
a, b, c = 1, 13, 17
n_list = [1000, 10000, 50000]
p_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
F = np.zeros((len(p_list), N))
f = np.zeros((len(n_list), N))
for i_p, p in enumerate(p_list):
    F[i_p] = calc_F(p)
for i_n, n in enumerate(n_list):
    f[i_n] = calc_f(n, a, b, c)
distr_spectr = pd.DataFrame(f.T, index=range(1, N + 1), columns=n_list)
distr_spectr.to_csv(dir_path + 'distr_spectr.csv')
distr_spectr

,1000,10000,50000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000500,0.000660
4,0.008000,0.001800,0.002440
5,0.007000,0.005500,0.005900
6,0.012000,0.014500,0.011320
7,0.024000,0.024300,0.021940
8,0.047000,0.034100,0.037380
9,0.069000,0.063100,0.065420
10,0.114000,0.098000,0.100260


In [194]:
res_arr = np.zeros((len(p_list), len(n_list)))
for i_n, n in enumerate(n_list):
    for i_p, p in enumerate(p_list):
        res_arr[i_p][i_n] = 1 - sum(f[i_n] * F[i_p])
df = pd.DataFrame(res_arr, index=p_list, columns=n_list)
df.to_csv(dir_path + 'distruction.csv')
df.round(6)

,1000,10000,50000
0.100000,0.000003,0.000003,0.000003
0.200000,0.000419,0.000419,0.000377
0.300000,0.006280,0.006829,0.006695
0.400000,0.044682,0.048195,0.048628
0.500000,0.181100,0.191769,0.192846
0.600000,0.451662,0.471632,0.471703
0.700000,0.754917,0.774633,0.774441
0.800000,0.938915,0.947999,0.948543
0.900000,0.993881,0.995964,0.995837
0.950000,0.999346,0.999645,0.999582


In [195]:
res_arr_monte = np.zeros([len(p_list), len(n_list)])
for i_n, n in enumerate(n_list):
    for i_p, p in enumerate(p_list):
        res_arr_monte[i_p][i_n] = count_res_monte(a, b, c, p, n)
df_monte = pd.DataFrame(res_arr_monte, index=p_list, columns=n_list)
df_monte = df_monte.round(6)
df_monte.to_csv(dir_path + 'monte_2.csv')
df_monte

,1000,10000,50000
0.100000,0.000000,0.000000,0.000020
0.200000,0.000000,0.000300,0.000500
0.300000,0.006000,0.006300,0.006740
0.400000,0.059000,0.050200,0.048960
0.500000,0.172000,0.186900,0.193500
0.600000,0.481000,0.467500,0.471160
0.700000,0.762000,0.772000,0.774600
0.800000,0.948000,0.943400,0.948820
0.900000,0.996000,0.995300,0.995320
0.950000,1.000000,0.999800,0.999540


In [196]:
def robust(p=0.95, n=1000):
    list_dictr, list_monte = np.zeros(10), np.zeros(10)
    for i in range(10):
        list_dictr[i] = 1 - sum(calc_f(n, a, b, c) * F[9])
        list_monte[i] = count_res_monte(a, b, c, p, n)
    df = pd.DataFrame([list_dictr, list_monte], index=['Distruction', 'Monte-Carlo'], columns=range(1, 11))
    df['robust'] = [np.std(list_dictr) / np.mean(list_dictr), np.std(list_monte) / np.mean(list_monte)]
    return df.T

In [197]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [198]:
robust_res = robust()
robust_res.to_csv(dir_path + 'robust.csv')
robust_res

,Distruction,Monte-Carlo
1,0.999741,0.999000
2,0.999583,1.000000
3,0.999719,1.000000
4,0.999276,0.998000
5,0.999545,1.000000
6,0.999482,0.999000
7,0.999677,1.000000
8,0.999905,1.000000
9,0.999318,1.000000
10,0.999474,1.000000
